In [1]:
import os
import numpy as np 
import cv2
import torch
import torch.nn as nn
from torchvision import models
from torch.autograd import Variable

In [20]:
def read_images(path, folders):
    images = []
    labels = []
    idx = 0
    for folder in folders:
        for filename in os.listdir(path+folder):
            image = cv2.imread(os.path.join(path+folder, filename))
            if image is not None:
                images.append(image)
                labels.append(idx)
                
        idx += 1
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

def getDescriptors(images, labels, model) : 
    features = []
    image_labels = []
    i = 0

    for image in images : 
        print (image.shape)
        # Re-sizing the image
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
        image = image.transpose(2, 1, 0)
        image = Variable(torch.Tensor(image))
        print (image.shape)
        image = image.unsqueeze(0)
        print (image.shape)
        feature = model(image)
        print (feature.data.shape)
        if feature is not None : 
            features.append(feature)
            image_labels.append(labels[i])
        i += 1
        print (feature)

        
    return features, image_labels

In [3]:
model = models.alexnet(pretrained=True)
new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
model.classifier = new_classifier
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
root = "/home/slr/Desktop/vr/Assignments/Feature-Learning/101_ObjectCategories/"

set_images = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]
images, labels = read_images("./../101_ObjectCategories/", set_images)

In [21]:
features, image_labels = getDescriptors(images, labels, model)

(283, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(218, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(258, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(300, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(300, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(300, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(299, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 

torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(283, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(203, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(300, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[52.5498,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<ReluBackward1>)
(248, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[80.8349,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<ReluBackward1>)
(283, 300, 3)
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 4096])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
(300, 297, 3)
torch.Size([3, 224, 224])
torch.Size([1

KeyboardInterrupt: 

In [17]:
# k = 50
# visDic = MiniBatchKMeans(init='k-means++',
#                             n_clusters=50,
#                             max_iter=1000,
#                             batch_size=1000, 
#                             n_init=10, 
#                             max_no_improvement=10, 
#                             verbose=0).fit(alexnet_des)


x_train, x_test, y_train, y_test = train_test_split(features, 
                                                    image_labels, 
                                                    test_size=0.1, 
                                                    random_state=4)


NameError: name 'features' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 284


In [ ]:
clf = cv2.ml.KNearest_create()
clf.train(x_train, cv2.ml.ROW_SAMPLE, np.asarray(y_train, dtype=np.float32))

In [ ]:
ret, results, neighbours ,dist = clf.findNearest(x_test, k=10)

In [ ]:
pred_label = []
for var in results:
    label = var
    pred_label.append(int(label))

print (y_test)
print (pred_label)
    
# Measuring the accuracies
metrics.accuracy_score(y_test, pred_label)